In [1]:
from TimewiseKfold import *
from Metrics import *
from InteractionDetection1 import *
import numpy as np

In [2]:
scaled_folds, PROVINCES_NUM = splitting_data(is_scaled=True, region=0)

features = ['Eva_I', 'Eva_II', 'Eva_III', 'Eva_IV', 'Eva_V', 'Eva_VI', 'Eva_VII', 'Eva_VIII', 'Eva_IX', 'Eva_X', 'Eva_XI', 'Eva_XII', 'Prcp_I', 'Prcp_II', 'Prcp_III', 'Prcp_IV', 'Prcp_V', 'Prcp_VI', 'Prcp_VII', 'Prcp_VIII', 'Prcp_IX', 'Prcp_X', 'Prcp_XI', 'Prcp_XII', 'Prcp_max_I', 'Prcp_max_II', 'Prcp_max_III', 'Prcp_max_IV', 'Prcp_max_V', 'Prcp_max_VI', 'Prcp_max_VII', 'Prcp_max_VIII', 'Prcp_max_IX', 'Prcp_max_X', 'Prcp_max_XI', 'Prcp_max_XII', 'Rdays_I', 'Rdays_II', 'Rdays_III', 'Rdays_IV', 'Rdays_V', 'Rdays_VI', 'Rdays_VII', 'Rdays_VIII', 'Rdays_IX', 'Rdays_X', 'Rdays_XI', 'Rdays_XII', 'Tavg_I', 'Tavg_II', 'Tavg_III', 'Tavg_IV', 'Tavg_V', 'Tavg_VI', 'Tavg_VII', 'Tavg_VIII', 'Tavg_IX', 'Tavg_X', 'Tavg_XI', 'Tavg_XII', 'Tmax_avg_I', 'Tmax_avg_II', 'Tmax_avg_III', 'Tmax_avg_IV', 'Tmax_avg_V', 'Tmax_avg_VI', 'Tmax_avg_VII', 'Tmax_avg_VIII', 'Tmax_avg_IX', 'Tmax_avg_X', 'Tmax_avg_XI', 'Tmax_avg_XII', 'Tmin_avg_I', 'Tmin_avg_II', 'Tmin_avg_III', 'Tmin_avg_IV', 'Tmin_avg_V', 'Tmin_avg_VI', 'Tmin_avg_VII', 'Tmin_avg_VIII', 'Tmin_avg_IX', 'Tmin_avg_X', 'Tmin_avg_XI', 'Tmin_avg_XII', 'Tmax_I', 'Tmax_II', 'Tmax_III', 'Tmax_IV', 'Tmax_V', 'Tmax_VI', 'Tmax_VII', 'Tmax_VIII', 'Tmax_IX', 'Tmax_X', 'Tmax_XI', 'Tmax_XII', 'Tmin_I', 'Tmin_II', 'Tmin_III', 'Tmin_IV', 'Tmin_V', 'Tmin_VI', 'Tmin_VII', 'Tmin_VIII', 'Tmin_IX', 'Tmin_X', 'Tmin_XI', 'Tmin_XII', 'Havg_I', 'Havg_II', 'Havg_III', 'Havg_IV', 'Havg_V', 'Havg_VI', 'Havg_VII', 'Havg_VIII', 'Havg_IX', 'Havg_X', 'Havg_XI', 'Havg_XII', 'Hmin_I', 'Hmin_II', 'Hmin_III', 'Hmin_IV', 'Hmin_V', 'Hmin_VI', 'Hmin_VII', 'Hmin_VIII', 'Hmin_IX', 'Hmin_X', 'Hmin_XI', 'Hmin_XII', 'Srad_I', 'Srad_II', 'Srad_III', 'Srad_IV', 'Srad_V', 'Srad_VI', 'Srad_VII', 'Srad_VIII', 'Srad_IX', 'Srad_X', 'Srad_XI', 'Srad_XII']

In [3]:
import collections

def union(a, b):
    res = list(a)
    
    for element in b:
        if (a.count(element) == 0):
            res.append(element)

    return res

def difference(a, b):
    res = list(a)
    
    for element in b:
        if (a.count(element) > 0):
            res.remove(element)
    
    return res 

def are_identical(list1, list2):
    return collections.Counter(list1) == collections.Counter(list2)
    

In [4]:
selected_features = ['Prcp_V', 'Prcp_IX', 'Tavg_II', 'Tmax_avg_XII', 'Srad_VIII']

alphas_res = []
deltas_res = []
rmse_train = 0
rmse_validation = 1.5243574378922755

round = 1

while(True):

    print("round", round)

    adding_lst = []
    removing_lst = []
    
    print('Adding Round:')
    for (i, feature) in enumerate(features):
        
        if (selected_features.count(feature) != 0):
            continue
        
        # if (not(i >= 79 and i <= 143)):
        #     continue
        print("The order of feature:", i, feature)
        
        new_selected_features = list(selected_features)
        new_selected_features.append(feature)
            
        alphas, deltas, rmse_train_cur, rmse_validation_cur = heuristic_interaction_detection(scaled_folds=scaled_folds, selected_features=new_selected_features, K=6, PROVINCES_NUM=PROVINCES_NUM)
            
        print(alphas, deltas, rmse_train_cur, rmse_validation_cur)
        
        if (rmse_validation_cur < rmse_validation):
            alphas_res = alphas
            deltas_res = deltas
            rmse_train = rmse_train_cur 
            rmse_validation = rmse_validation_cur
            
            
            adding_lst.append(feature)
    
    # break
    
    # adding_lst = ['Eva_II', 'Eva_III', 'Eva_IV', 'Prcp_II', 'Rdays_VII', 'Tmax_avg_X']
    
    print('Removing Round:')
    for (i, feature) in enumerate(selected_features):
        
        # if(not(i > 10)):
        #     continue
        
        print("The order of feature:", i, feature)
        
        new_selected_features = difference(union(selected_features, adding_lst), [feature])
        
        alphas, deltas, rmse_train_cur, rmse_validation_cur = heuristic_interaction_detection(scaled_folds=scaled_folds, selected_features=new_selected_features, K=6, PROVINCES_NUM=PROVINCES_NUM)
        
        print(alphas, deltas, rmse_train_cur, rmse_validation_cur)
         
        if (rmse_validation_cur < rmse_validation):
            alphas_res = alphas
            deltas_res = deltas
            rmse_train = rmse_train_cur 
            rmse_validation = rmse_validation_cur
                
            # print(alphas_res, deltas_res, rmse_train, rmse_validation)
            removing_lst.append(feature)
    
    # break
    
    new_selected_features = difference(union(selected_features, adding_lst), removing_lst)
    
    if (are_identical(selected_features, new_selected_features)):
        break
    else:
        selected_features = new_selected_features
        # alphas_res, deltas_res, rmse_train, rmse_validation = heuristic_interaction_detection(scaled_folds=scaled_folds, selected_features=selected_features, K=6, PROVINCES_NUM=PROVINCES_NUM)
        # print('New RMSE validation', rmse_validation)
        print('adding list', adding_lst)
        print('removing list', removing_lst)
        print('new selected features', selected_features)
    round += 1

round 1
Adding Round:
The order of feature: 0 Eva_I
[array([1. , 0.5, 0.5, 0. , 0.5, 0.5]), array([0.5, 0. , 0.5, 0.5, 0. , 0.5])] [array([0., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0., 0.])] 1.5467865211440965 1.547255789124506
The order of feature: 1 Eva_II
[] [] 1.653463328076403 1.331802725884492
The order of feature: 2 Eva_III
[array([1. , 0.5, 0.5, 0. , 0.5, 0.5]), array([0.5, 0. , 0.5, 0.5, 0. , 0.5])] [array([0., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0., 0.])] 1.543388414573304 1.5022103258539363
The order of feature: 3 Eva_IV
[array([1. , 0.5, 0.5, 0. , 0.5, 0.5]), array([0.5, 0. , 0.5, 0.5, 0. , 0.5])] [array([0., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0., 0.])] 1.5477982512566453 1.5194847240386933
The order of feature: 4 Eva_V
[array([1. , 0.5, 0.5, 0. , 0.5, 0.5]), array([0.5, 0. , 0.5, 0.5, 0. , 0.5]), array([0.5, 0.5, 0.5, 1. , 0.5, 0. ])] [array([0., 0., 0., 0., 1., 0.]), array([0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0.])] 1.480680118364